In [1]:
!git clone https://github.com/andreinechaev/nvcc4jupyter.git
!pip install ./nvcc4jupyter
%load_ext nvcc4jupyter

Cloning into 'nvcc4jupyter'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 507 (delta 200), reused 282 (delta 159), pack-reused 121
Receiving objects: 100% (507/507), 106.61 KiB | 2.22 MiB/s, done.
Resolving deltas: 100% (238/238), done.
Processing ./nvcc4jupyter
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-c6ex3da5/wheels/40/0b/4b/bf52a1a48e3fb52ba03f8509612c6989e87d8fab1f8b41408f
Successfully built nvcc4jupyter
Detected platform "Kaggle". Running its setup...
Updating the package lists...
Installing nvidia-cuda-toolkit, this may take a few minutes...
Source files will be saved in 

In [3]:
# Python3 program to implement traveling salesman 
# problem using naive approach. 
import sys
from sys import maxsize 
import random
from itertools import permutations
V = 4
debug = False
# implementation of traveling Salesman Problem 
def travellingSalesmanProblem(graph, s): 

	# store all vertex apart from source vertex 
	vertex = [] 
	for i in range(V): 
		if i != s: 
			vertex.append(i) 

	# store minimum weight Hamiltonian Cycle 
	min_path = maxsize 
	next_permutation=permutations(vertex)
	# print all permutations

	for i in next_permutation:
		# print(i)
		# store current Path weight(cost) 
		current_pathweight = 0
        
        
		# compute current path weight 
		k = s 
		
		for j in i: 
			current_pathweight += graph[k][j] 
			k = j 
		current_pathweight += graph[k][s] 

		# update minimum 
		min_path = min(min_path, current_pathweight) 
		
	return min_path 
def test_generator(n, filename):
    with open(filename, 'w') as f:
        f.write(f"{n}\n")
        for u in range(n):
            for v in range(u + 1, n):
                w = random.uniform(1, 100)
                f.write(f"{u} {v} {w:.2f}\n")

def read_edges_from_file(filename):
    edges = []
    with open(filename, 'r') as f:
        # Skip the first line
        next(f)
        for line in f:
            u, v, w = line.split()
            edges.append((int(u), int(v), float(w)))
    return edges

def create_graph(n, edges):
    graph = [[0 if i == j else sys.maxsize for j in range(n)] for i in range(n)]
    for u, v, w in edges:
        graph[u][v] = w
        graph[v][u] = w
    return graph

# Driver Code 
if __name__ == "__main__": 
    V = 5
    filename = "edges.txt"

    # Generate a random test case and save to file
    test_generator(V, filename)

    # Read the edges from the file
    edges = read_edges_from_file(filename)
    if debug:
        print("Read Edges from file:")
        for u, v, w in edges:
            print(f"{u} {v} {w:.2f}")

    # Create a graph from the edges
    graph = create_graph(V, edges)
    if debug:
        print("Graph:")
        for vec in graph:
            print(vec)
	# set the sol to maxmimum value of int 
    sol = sys.maxsize
	# print(travellingSalesmanProblem(graph, s))
    for s in range(V):
     	sol=min(sol,travellingSalesmanProblem(graph, s))
    print("the best solution is : ",sol)

the best solution is :  352.34999999999997


In [7]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <float.h>
#include <math.h>
#include <cuda.h>


// CUDA kernel to generate the k-th permutation
__global__ void kthPermutation(int n, int i, int *fact, int *perm) {
    int j, k = 0;
    for (k = 0; k < n; ++k) {
        perm[k] = i / fact[n - 1 - k];
        i = i % fact[n - 1 - k];
    }
    // Adjust values to obtain the permutation
    for (k = n - 1; k > 0; --k)
        for (j = k - 1; j >= 0; --j)
            if (perm[j] <= perm[k])
                perm[k]++;
}

// CUDA kernel to calculate the path weight for a given permutation
__global__ void calculatePathWeight(double *graph, int *fact, double *min_path, int numPerms, int numVertices) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= numPerms) return;

    int *path = new int[numVertices];
    kthPermutation<<<1, 1>>>(numVertices, idx, fact, path);

    double current_pathweight = 0;
    for (int j = 1; j < numVertices; j++) {
        int u = path[j - 1];
        int v = path[j];
        current_pathweight += graph[u * numVertices + v];
    }
    int s = path[0];
    int k = path[numVertices - 1];
    current_pathweight += graph[k * numVertices + s];
    if(current_pathweight<min_path[0]){
        min_path[0] = current_pathweight;
    }
    //atomicMin_double(min_path, current_pathweight);

    delete[] path;
}
// Function to read the graph from a file
void readGraphFromFile(FILE *file, double** graph) {
    if (file == NULL) {
        fprintf(stderr, "Error opening file\n");
        exit(EXIT_FAILURE);
    }

    int u, v;
    double w;
    while (fscanf(file, "%d %d %lf", &u, &v, &w) == 3) {
        graph[u][v] = w;
        graph[v][u] = w;
    }

    fclose(file);
}
int main(int argc, char **argv) {
    char*file_name;
    // if (argc != 2) {
    //     fprintf(stderr, "Usage: %s <filename>\n", argv[0]);
    //     return EXIT_FAILURE;
    // }
    // file_name = argv[1];
    file_name="edges.txt";
    FILE *file = fopen(file_name, "r");
    if (file == NULL) {
        fprintf(stderr, "Error opening file\n");
        exit(EXIT_FAILURE);
    }
    int N;
    fscanf(file,"%d",&N);
    double** graph = (double**)malloc(N * sizeof(double*));
    for(int i=0;i<N;i++){
        graph[i]=(double*)malloc(N * sizeof(double));
    }
    // Initialize the graph with 0s for diagonal and infinity for other entries
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            if (i == j) {
                graph[i][j] = 0;
            } else {
                graph[i][j] = DBL_MAX;
            }
        }
    }
    // Read the graph from the file
    readGraphFromFile(file, graph);

    double *h_graph = (double *)malloc(N * N * sizeof(double));
    double *d_graph;

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            h_graph[i * N + j] = graph[i][j];
        }
    }
    cudaMalloc(&d_graph, N * N * sizeof(double));
    cudaMemcpy(d_graph, h_graph, N * N * sizeof(double), cudaMemcpyHostToDevice);

    int *h_fact = (int *)malloc((N + 1) * sizeof(int));
    int *d_fact;
    h_fact[0] = 1;
    for (int i = 1; i <= N; i++) {
        h_fact[i] = h_fact[i - 1] * i;
    }
    cudaMalloc(&d_fact, (N + 1) * sizeof(int));
    cudaMemcpy(d_fact, h_fact, (N + 1) * sizeof(int), cudaMemcpyHostToDevice);

    double *d_min_path;
    cudaMalloc(&d_min_path, sizeof(double));
    double h_min_path = DBL_MAX;
    cudaMemcpy(d_min_path, &h_min_path, sizeof(double), cudaMemcpyHostToDevice);

    int numPerms = h_fact[N];
    int blockSize = 256;
    int numBlocks = (numPerms + blockSize - 1) / blockSize;
    calculatePathWeight<<<numBlocks, blockSize>>>(d_graph, d_fact, d_min_path, numPerms, N);

    cudaMemcpy(&h_min_path, d_min_path, sizeof(double), cudaMemcpyDeviceToHost);
    printf("Minimum Path Weight: %lf\n", h_min_path);

    cudaFree(d_graph);
    cudaFree(d_fact);
    cudaFree(d_min_path);
    free(h_graph);
    free(h_fact);

    return 0;
}


/tmp/tmpwljpktdf/8005a49e-9a97-4db9-9944-06d2ac67eee6/single_file.cu(69): warning #2464-D: conversion from a string literal to "char *" is deprecated
      file_name="edges.txt";
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/tmp/tmpwljpktdf/8005a49e-9a97-4db9-9944-06d2ac67eee6/single_file.cu(28): error: kernel launch from __device__ or __global__ functions requires separate compilation mode
      kthPermutation<<<1, 1>>>(numVertices, idx, fact, path);
      ^

/tmp/tmpwljpktdf/8005a49e-9a97-4db9-9944-06d2ac67eee6/single_file.cu(69): warning #2464-D: conversion from a string literal to "char *" is deprecated
      file_name="edges.txt";
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

1 error detected in the compilation of "/tmp/tmpwljpktdf/8005a49e-9a97-4db9-9944-06d2ac67eee6/single_file.cu".



In [ ]:
# Note: essential compilation flags for cdp
!nvcc kernal.cu -o run -rdc=true -lcudadevrt
!nvprof ./run float_numbers.txt 1.28

In [ ]:
!pwd
